# Retrieval augmented generation (RAG)

In [1]:
# Download a file
import wget

file_path = "my_document.pdf"
doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

wget.download(doc_url, out=file_path)

-1 / unknown

'my_document.pdf'

In [2]:
# Request configuration
import requests

base_url = "http://localhost:8080"
api_key = "albert" # enter any value if no api_key is setup in config.ini file

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}  # skip headers if no api_key is setup in config.ini file

In [3]:
# Upload a file
import os

me = "leo"
model = "BAAI/bge-m3"

files = {'files': (os.path.basename(file_path), open(file_path, 'rb'), "application/pdf")}
response = session.post(f"{base_url}/files?user={me}&model={model}", files=files)

response.content

b'{"object":"list","data":[{"object":"upload","id":"b7b3901d-6b36-47ed-9643-686d1574cabd","name":"my_document.pdf","status":"success"}]}'

In [4]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{me}")

response.content

b'{"object":"list","data":[{"object":"file","id":"7ca4b017-fcb9-4e10-b5c0-d810507d80a0","name":"my_document.pdf","vector_ids":["ba9eb2f5-ea4f-4949-9d61-dc02e9b530c3","fbbdcc6a-6e34-40bf-ad81-414ffb79764c"]},{"object":"file","id":"b7b3901d-6b36-47ed-9643-686d1574cabd","name":"my_document.pdf","vector_ids":["920c6b40-cd8b-4db6-bd6d-aaef831ad964","bb36bcb1-f006-4f5f-af4d-d86ecdb013ad"]},{"object":"file","id":"d127bf8a-f0b0-4259-bd3f-ea2954bdbff6","name":"my_document.pdf","vector_ids":["050e81ba-da23-4716-8300-d6313796c4d4","9c17bdae-1dae-4fd9-a23f-dbb26b1e4e0e"]}],"metadata":{"files":3,"vectors":6}}'

In [5]:
file_id = response.json()["data"][0]["id"]

In [6]:
# Get tools
response = session.get(f"{base_url}/tools")
response.content

b'{"object":"list","data":[{"id":"BaseRAG","description":"Base RAG, basic retrival augmented generation.\\n\\nArgs:\\n    embeddings_model (str): OpenAI embeddings model\\n    collection_name (Optional[str], optional): Collection name. Defaults to \\"user\\" parameters.\\n    file_ids (Optional[List[str]], optional): List of file ids. Defaults to None.\\n    k (int, optional): Top K. Defaults to 4.\\n    prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.","object":"tool"}]}'

In [7]:
# Display tools parameters
print(response.json()["data"][0]["description"])

Base RAG, basic retrival augmented generation.

Args:
    embeddings_model (str): OpenAI embeddings model
    collection_name (Optional[str], optional): Collection name. Defaults to "user" parameters.
    file_ids (Optional[List[str]], optional): List of file ids. Defaults to None.
    k (int, optional): Top K. Defaults to 4.
    prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.


In [8]:
# OpenAI client configuration
from openai import OpenAI

client = OpenAI(base_url=f"{base_url}/v1", api_key=api_key)

In [9]:
# Chat completions
data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Qui est Ulrich Tan ?"}],
    "stream": False,
    "n": 1,
    "user": "leo",
    "tools": [
        {
            "function": {
                "name": "BaseRAG",
                "parameters": {"embeddings_model": "BAAI/bge-m3", "k": 2}
            },
            "type": "function"
        }
    ]
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

Selon les documents, Ulrich Tan est le chef du pôle Datamin du département "Etalab".


In [10]:
# View chat history
chat_id = response.id

response = session.get(url=f"{base_url}/chat_history/{me}/{chat_id}")
print(response.json()["messages"][0]["content"])

Réponds à la question suivante en te basant sur les documents ci-dessous : Qui est Ulrich Tan ?

Documents :

– Mme Marine Boudeau, agente contractuelle, préfiguratrice du pôle « Brigade d’intervention numérique, du département « ACE »;– M. Mohsen Souissi, agent contractuel, préfigurateur du pôle « Référentiels, capitalisation et observatoires  du département « ACE »;– M. Louis di Benedetto, inspecteur en chef de la santé publique vétérinaire, chef du pôle « Pilotage de la qualité et des partenariats  du département « ISO »;– M. Philippe Levillain, agent contractuel, chef du pôle « Réseau interministériel de l’Etat / SOI » du département « ISO »;– M. Gilles Corbi, agent contractuel, chef du pôle Production du département « ISO »;– M. Yann Brûlé, grade, chef du pôle « SI RIE  du département « ISO »;– M. Ulrich Tan, chef du pôle Datamin du département « Etalab »;– Mme Marie-Christie Ritz, cheffe du pôle « RH et attractivité »;– Mme Fadila Leturcq, cheffe du pôle « Campus du numérique ».A